In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import Formatter
import datetime as dt
import pandas as pd

from stable_baselines.common.policies import MlpLstmPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.ppo2 import PPO2
from stable_baselines import bench, logger
from stable_baselines.results_plotter import load_results, ts2xy

from importlib import reload
from time import time
import os.path

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [42]:
train = pd.read_csv("train.csv", index_col=0, parse_dates=True, dayfirst=True)

In [65]:
train.rename(columns=lambda x: x.strip(), inplace=True)

In [43]:
train

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS,Cash
2015-01-01,40.94,53.630,159.28,110.38,65.44,90.16,38.06,45.02,89.31,72.70,...,43.290,92.45,53.27,36.5155,38.07,52.3831,113.42,28.510,43.03,0.0355
2015-01-02,40.56,53.910,158.56,109.33,65.89,90.46,37.31,44.90,88.84,72.34,...,43.600,92.83,51.93,36.2257,38.08,52.0236,112.59,28.290,43.31,0.0152
2015-01-05,39.80,53.875,156.47,106.25,64.65,89.69,37.07,44.91,87.34,71.98,...,42.795,90.29,51.57,35.4353,35.71,50.9666,116.79,27.230,43.05,0.0101
2015-01-06,39.18,53.040,156.36,106.26,64.33,90.18,36.13,44.40,86.71,70.53,...,42.180,89.81,50.93,34.9611,35.50,50.3410,115.80,26.190,42.63,0.0253
2015-01-07,39.70,53.010,159.72,107.75,66.93,91.98,37.28,44.76,88.53,71.11,...,42.195,90.72,52.25,35.4089,35.78,52.0092,118.68,26.435,43.51,0.0203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-25,63.15,44.910,124.91,188.58,101.08,85.49,394.28,62.37,155.99,243.56,...,69.600,78.71,46.57,27.9100,71.40,82.5300,111.08,57.050,83.67,1.8891
2018-05-28,63.15,44.910,124.91,188.58,101.08,85.49,394.28,62.37,155.99,243.56,...,69.600,78.71,46.57,27.9100,71.40,82.5300,111.08,57.050,83.67,1.8891
2018-05-29,61.85,43.590,123.96,187.90,99.47,82.57,387.40,61.30,153.54,242.19,...,68.230,78.42,45.81,27.7500,70.55,81.9100,109.27,54.920,83.15,1.8737
2018-05-30,62.69,43.300,128.53,187.50,103.01,84.13,388.57,62.07,156.03,247.40,...,68.020,81.50,45.45,27.7800,71.49,82.5100,110.65,55.530,84.45,1.9127


In [44]:
na_col = train.columns[pd.isnull(train).sum() != 0]

In [45]:
train.shape

(891, 506)

In [52]:
yest = np.roll(train, 1, axis=0)
yest[0] = np.nan

In [67]:
ret1 = (train - yest) / yest

In [27]:
pd.datetime(2015,1,13) < pd.datetime(2015,1,13)

True

In [58]:
train.index[(train.index > pd.datetime(2017, 1, 1)) & (train.index < pd.datetime(2017, 8, 1))]

DatetimeIndex(['2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01',
               '2017-06-01', '2017-01-13', '2017-01-16', '2017-01-17',
               '2017-01-18', '2017-01-19',
               ...
               '2017-01-11', '2017-02-11', '2017-03-11', '2017-06-11',
               '2017-07-11', '2017-01-12', '2017-04-12', '2017-05-12',
               '2017-06-12', '2017-07-12'],
              dtype='datetime64[ns]', length=152, freq=None)

In [45]:
train[na_col]

,AMCR,AMD,ARNC,CSX,CTVA,DOW,DXC,EXC,FOX,FOXA,...,PEP,PFG,PYPL,QRVO,REG,TMUS,UA,WLTW,WRK,XEL
2015-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-01,NaN,2.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,70.40,NaN,NaN,NaN,NaN,NaN,NaN
2015-05-01,NaN,2.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,67.63,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-01,NaN,2.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,64.67,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-01,NaN,2.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,66.65,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-25,NaN,13.54,18.43,64.68,NaN,NaN,81.4864,NaN,NaN,NaN,...,100.31,58.23,80.96,81.49,NaN,56.59,18.88,155.26,59.96,45.19
2018-05-28,NaN,13.54,18.43,64.68,NaN,NaN,81.4864,NaN,NaN,NaN,...,100.31,58.23,80.96,81.49,NaN,56.59,18.88,155.26,59.96,45.19
2018-05-29,NaN,13.36,17.93,64.22,NaN,NaN,77.8363,NaN,NaN,NaN,...,101.10,55.24,80.73,79.94,NaN,56.62,18.57,150.75,59.03,45.25
2018-05-30,NaN,13.82,18.12,65.64,NaN,NaN,80.1544,NaN,NaN,NaN,...,101.83,56.36,81.60,80.93,NaN,56.33,19.20,152.90,59.76,45.61


In [46]:
for c in na_col:
    print(c, train[c].isnull().sum())

AMCR 891
AMD 1
ARNC 478
CSX 253
CTVA 891
DOW 891
DXC 575
EXC 891
FOX 891
FOXA 891
FTV 377
HPE 207
INFO 864
KHC 132
LNT 891
LW 478
NCLH 773
NWL 891
PEP 774
PFG 772
PYPL 132
QRVO 1
REG 891
TMUS 213
UA 319
WLTW 263
WRK 124
XEL 783
